# ⚙️ 3.1 – Patrón Creacional: Factory Method

El patrón **Factory Method** proporciona una forma flexible de crear objetos **sin acoplar el código cliente a clases concretas**.

Se basa en delegar la lógica de creación de objetos a subclases o métodos especializados, favoreciendo el principio **OCP (Open/Closed)**.

---
## 🎯 Objetivos
- Comprender el propósito del patrón Factory Method.
- Implementar el patrón en TypeScript.
- Comparar su uso frente a la creación directa de objetos.
- Aplicar el principio de extensión sin modificación (OCP).

In [1]:
console.log('✅ Notebook 3.1 – Factory Method listo para usar.');

✅ Notebook 3.1 – Factory Method listo para usar.


---
## 1️⃣ Problema inicial

Supongamos que estamos desarrollando una aplicación que genera distintos tipos de reportes: **PDF** y **HTML**.
Sin aplicar el patrón, el código podría depender directamente de las clases concretas:

In [ ]:
class PDFReport {
  generate() { console.log('📄 Generando reporte en PDF'); }
}

class HTMLReport {
  generate() { console.log('🌐 Generando reporte en HTML'); }
}

class XMLReport {
  generate() { console.log('🌐 Generando reporte en XML'); }
}

class ReportGenerator {
  createReport(type: string) {
    if (type === 'pdf') return new PDFReport();
    if (type === 'html') return new HTMLReport();
    if (type === 'xml') return new XMLReport()
    throw new Error('Tipo de reporte no soportado');
  }
}

const factory = new ReportGenerator();
factory.createReport('pdf').generate();

🔍 Este enfoque **viola OCP**, ya que cada vez que agreguemos un nuevo tipo de reporte, debemos **modificar la clase** `ReportGenerator`.

Para solucionarlo, aplicaremos el **Factory Method**.

---
## 2️⃣ Aplicando el patrón Factory Method

Definimos una **interfaz común** para los productos (reportes) y una clase base con el **método fábrica** que delegará la creación a las subclases.

In [2]:
interface Report {
  generate(): void;
}

class PDFReport implements Report {
  generate() { console.log('📄 Generando reporte en PDF'); }
}

class HTMLReport implements Report {
  generate() { console.log('🌐 Generando reporte en HTML'); }
}

class XMLReport implements Report {
  generate() { console.log('🌐 Generando reporte en XML'); }
}

abstract class ReportCreator {
  // Método fábrica que será implementado por las subclases
  abstract createReport(): Report;

  // Lógica común que usa el producto creado
  generateReport() {
    const report = this.createReport();
    console.log('🧩 Preparando entorno...');
    report.generate();
    console.log('✅ Reporte completado');
  }
}

class PDFReportCreator extends ReportCreator {
  createReport(): Report { return new PDFReport(); }
}

class HTMLReportCreator extends ReportCreator {
  createReport(): Report { return new HTMLReport(); }
}

class XMLReportCreator extends ReportCreator {
  createReport(): Report { return new XMLReport() }
}

const pdfCreator = new PDFReportCreator();
pdfCreator.generateReport();

const htmlCreator = new HTMLReportCreator();
htmlCreator.generateReport();

🧩 Preparando entorno...
📄 Generando reporte en PDF
✅ Reporte completado
🧩 Preparando entorno...
🌐 Generando reporte en HTML
✅ Reporte completado


✅ Ahora el código cumple **OCP**: si queremos agregar un nuevo tipo de reporte (por ejemplo, *ExcelReport*), solo necesitamos **añadir una nueva subclase**, sin modificar las existentes.

---
## 3️⃣ Diagrama conceptual (simplificado)

```mermaid
classDiagram
    class ReportCreator {
      +createReport(): Report
      +generateReport()
    }
    class PDFReportCreator
    class HTMLReportCreator
    class Report {
      <<interface>>
      +generate()
    }
    class PDFReport
    class HTMLReport

    ReportCreator <|-- PDFReportCreator
    ReportCreator <|-- HTMLReportCreator
    Report <|.. PDFReport
    Report <|.. HTMLReport
```

---
## 4️⃣ 🧩 Ejercicio práctico – Nuevo tipo de reporte

Crea una nueva clase `ExcelReport` y su correspondiente `ExcelReportCreator` que genere mensajes como:

```
📊 Generando reporte en Excel
```

Luego, instancia el nuevo creador y ejecuta `generateReport()`.

In [ ]:
// Escribe tu código aquí...

### ✅ Solución propuesta

In [3]:
class ExcelReport implements Report {
  generate() { console.log('📊 Generando reporte en Excel'); }
}

class ExcelReportCreator extends ReportCreator {
  createReport(): Report { return new ExcelReport(); }
}

const excelCreator = new ExcelReportCreator();
excelCreator.generateReport();

🧩 Preparando entorno...
📊 Generando reporte en Excel
✅ Reporte completado


In [ ]:
class EmailReport implements Report {
  generate() { console.log('📊 Generando reporte en Excel'); }
}

class EmailReportCreator extends ReportCreator {
  createReport(): Report { return new EmailReport(); }
}

const emailCreator = new EmailReportCreator();
emailCreator.generateReport();

---
## 5️⃣ Cuándo usar Factory Method

### ✅ Úsalo cuando:
- No sepas de antemano qué clase de objeto necesitas crear.
- Quieras delegar la creación de objetos a subclases.
- Desees aplicar el principio **OCP** (extender sin modificar).

### 🚫 Evítalo cuando:
- El sistema tenga pocos tipos de objetos y la creación sea simple.
- Se requiera un único punto de acceso (usa *Singleton* en su lugar).

💡 En la práctica, muchos frameworks usan *Factory Methods* para construir controladores, servicios o componentes dinámicamente.

---
## 🧠 Resumen del notebook

- El patrón **Factory Method** delega la creación de objetos a subclases.
- Facilita la extensión sin modificar código existente (**OCP**).
- Se usa ampliamente en frameworks y librerías.

➡️ Próximo notebook → **3.2 Abstract Factory y Builder**.

# EJEMPLO:  General Service Factory para una app modular

🎯 Objetivo

* Poder registrar servicios dinámicamente (autenticación, reporting, notificaciones, logs, etc.).

* Obtenerlos en tiempo de ejecución sin acoplarse a sus clases concretas.

* Permitir añadir nuevos servicios sin tocar el código existente (OCP).

## Definimos la interfaz generica de lo que es UN SERVICIO


In [ ]:
export interface Service {
  execute(...args: any[]): any;
}


## Creamos la Factory general de servicios + Singleton


In [15]:
// capaz de crear instancias de servicios, entregarlas y almacenarlas
class ServiceFactory {
    private static services = new Map<string, new (...args:any[]) => Service>()
    private static instances = new Map<string, Service>()
    
    static register(key:string, serviceClass: new () => Service) {
        this.services.set(key,serviceClass);
    }
    // Obtener una instancia de servicio
    static get(key:string): Service {
        if(this.instances.has(key)){
            return this.instances.get(key)!;
        }

        const ServiceClass = this.services.get(key)


        if(!ServiceClass) {
            throw new Error(`Servicio ${key} no esta registrado en esta factory`)
        }
        const instance = new ServiceClass()
        this.instances.set(key,instance)
        return instance
    }
}


In [16]:
class EmailService implements Service {
  execute(to: string, msg: string) {
    console.log(`📨 Enviando email a ${to}: ${msg}`);
  }
}

class LogService implements Service {
  execute(msg: string) {
    console.log(`📝 LOG: ${msg}`);
  }
}

class BillingService implements Service {
  execute(amount: number) {
    console.log(`💰 Procesando pago de ${amount}€`);
  }
}



In [17]:
ServiceFactory.register("email",EmailService);
ServiceFactory.register("log", LogService);
ServiceFactory.register("billing", BillingService);

In [18]:
const email = ServiceFactory.get("email");
email.execute("cliente@dominio.com", "Tu pedido está listo");

const log = ServiceFactory.get("log");
log.execute("Arrancando sistema...");

const billing = ServiceFactory.get("billing");
billing.execute(199.99);


📨 Enviando email a cliente@dominio.com: Tu pedido está listo
📝 LOG: Arrancando sistema...
💰 Procesando pago de 199.99€
